**Multiclass Fish Image Classification using Pre-trained models**

**InceptionV3**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import os
from PIL import Image
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix

In [ ]:
# set the device
#This means you’ll run your model on GPU

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# transformation
#InceptionV3 expects images of size 299x299.
# for training data

train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(299, scale = (0.8, 1.0)), #random zoom + crop
        transforms.RandomHorizontalFlip(), # flip the image (right ---> left)
        transforms.RandomRotation(15), # random rotation clockwise or counter clockwise
        transforms.ToTensor() #rescale to [0,1]

    ])

In [ ]:
# for test and val
# data preprocessing

test_val_transform = transforms.Compose(
    [
        transforms.Resize((299, 299)), #resize piexel same as training
        transforms.ToTensor() #convert image to tensor
    ]
)

In [ ]:

# load the dataset

train_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/train', transform = train_transform)

test_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/test', transform= test_val_transform)

val_datasets = datasets.ImageFolder('/content/drive/MyDrive/images.cv_jzk6llhf18tm3k0kyttxz/data/val', transform= test_val_transform)


In [ ]:
# create the dataloader
train_data_loader = DataLoader(train_datasets, batch_size = 5, shuffle = True)
test_data_loader = DataLoader(test_datasets, batch_size = 5, shuffle = True)
val_data_loader = DataLoader(val_datasets, batch_size = 5, shuffle = True)


In [ ]:
# load the pre-trained  model ResNet50
model = models.inception_v3(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 197MB/s] 


In [ ]:
model.fc = nn.Linear(model.fc.in_features, 11)
model = model.to(device)

In [ ]:
#loss function and optimizer

cl = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# train the model

epochs = 25

for i in range(epochs):
  model.train()
  total_loss = 0.0 # total running loss
  total = 0 #  #  Total number of images evaluated
  correct = 0 #  calculate how many predictions correct

  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    optimizer.zero_grad()
    outputs = model(img)
    output_main = outputs.logits  ## InceptionV3 return InceptionOutputs which includes two outputs during training
    loss = cl(output_main, tar)

    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    _, preds = torch.max(output_main, 1)
    correct += (preds == tar).sum().item()
    total += tar.size(0)

  print(f"Epoch [{i + 1}/{epochs}], loss {total_loss:.4f}")


Epoch [1/25], loss 824.3374
Epoch [2/25], loss 721.5575
Epoch [3/25], loss 693.1239
Epoch [4/25], loss 663.9621
Epoch [5/25], loss 676.7073
Epoch [6/25], loss 653.0401
Epoch [7/25], loss 630.0506
Epoch [8/25], loss 637.2027
Epoch [9/25], loss 663.4692
Epoch [10/25], loss 677.6671
Epoch [11/25], loss 663.7045
Epoch [12/25], loss 694.9322
Epoch [13/25], loss 672.9573
Epoch [14/25], loss 694.4688
Epoch [15/25], loss 669.0194
Epoch [16/25], loss 698.7586
Epoch [17/25], loss 677.7195
Epoch [18/25], loss 710.9324
Epoch [19/25], loss 733.1925
Epoch [20/25], loss 703.2789
Epoch [21/25], loss 645.4709
Epoch [22/25], loss 668.6607
Epoch [23/25], loss 685.3732
Epoch [24/25], loss 703.9638
Epoch [25/25], loss 681.7332


In [ ]:
model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in val_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)


    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  #accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"val_accuracy: {accuracy*100 :.2f}%")

  #precision
  precision = precision_score(all_tars , all_preds, average = 'macro')
  print(f"val_precision: {precision*100 :.2f}%")

  # recall
  recall = recall_score(all_tars , all_preds, average = 'macro')
  print(f"val_recall: {recall*100 :.2f}%")


#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"val f1_score: {f1score *100:.2f}%")

# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

val_accuracy: 96.43%
val_precision: 96.79%
val_recall: 93.89%
val f1_score: 94.97%
confusion matrix
[[186   0   0   0   0   0   0   1   0   0   0]
 [  3   7   0   0   0   0   0   0   0   0   0]
 [  0   0 105   0   0   0   0   0   0   0   0]
 [  0   0   0  80   2   0   1   9   0   0   2]
 [  0   0   0   0  97   0   0   0   0   0   0]
 [  0   0   0   0   0  80   0   0   0  10   0]
 [  0   0   0   1   0   0 110   1   0   1   0]
 [  0   0   2   0   0   0   0  95   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0]
 [  0   0   1   0   1   0   0   2   0  97   0]
 [  0   0   0   0   0   0   0   2   0   0  96]]


In [ ]:
# Evaluation on test data

model.eval()

all_preds = []
all_tars = []

with torch.no_grad():
  for img, tar in test_data_loader:

    img, tar = img.to(device), tar.to(device)

    outputs = model(img)
    _, preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


# accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Test Accuracy: {accuracy *100:.2f}%")

# precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Test precision: {precision *100:.2f}%")

#recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Test recall: {recall *100:.2f}%")


#f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f"Test f1_score: {f1score *100:.2f}%")

# Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)


Test Accuracy: 95.29%
Test precision: 93.37%
Test recall: 93.52%
Test f1_score: 93.20%
confusion matrix
[[513   4   2   0   0   0   0   1   0   0   0]
 [  3  10   0   0   0   0   0   0   0   0   0]
 [  0   0 298   0   0   0   0   0   0   0   0]
 [  1   0   2 233   8   0  22  36   0   0   3]
 [  0   0   3   0 283   0   0   0   0   0   0]
 [  0   0   1   0   3 254   0   0   0  33   0]
 [  0   0   0   1   0   0 267   4   0   1   0]
 [  1   0   4   1   0   0   0 319   0   1   1]
 [  1   0   0   0   0   0   0   0 287   1   0]
 [  0   0   4   0   0   0   0   3   0 286   0]
 [  0   0   0   0   0   0   0   5   0   0 287]]


In [ ]:
# evaluate the train data

model.eval()

all_preds = []
all_tars = []


with torch.no_grad():
  for img, tar in train_data_loader:
    img, tar = img.to(device), tar.to(device)
    outputs = model(img)
    _,preds = torch.max(outputs, 1)

    all_preds.extend(preds.cpu().numpy())
    all_tars.extend(tar.cpu().numpy())


  # accuracy
  accuracy = accuracy_score(all_tars, all_preds)
  print(f"Train Accuracy: {accuracy* 100:.2f}%")

  #precision
  precision = precision_score(all_tars, all_preds, average = 'macro')
  print(f"Train precision: {precision*100:.2f}%")

  #recall
  recall = recall_score(all_tars, all_preds, average = 'macro')
  print(f"Train recall:{recall*100:.2f}%")


  #f1 score
  f1score = f1_score(all_tars, all_preds, average = 'macro')
  print(f" f1_score: {f1score *100:.2f}%")

  # Calculate confusion matrix
  cm = confusion_matrix(all_tars, all_preds)
  print('confusion matrix')
  print(cm)

Train Accuracy: 97.98%
Train precision: 97.18%
Train recall:97.18%
 f1_score: 97.16%
confusion matrix
[[1087    3    2    0    0    1    0    2    0    1    0]
 [   3   27    0    0    0    0    0    0    0    0    0]
 [   0    0  569    0    0    0    0    0    0    0    0]
 [   0    0    2  517    2    0    7   30    0    0    8]
 [   1    0    6    0  563    1    0    0    0    2    0]
 [   0    0    2    0    1  557    0    0    1   18    0]
 [   1    0    0    5    0    0  562    2    0    1    0]
 [   1    0    5    1    0    0    0  526    0    1    4]
 [   0    0    0    0    0    0    0    0  576    0    0]
 [   0    0    7    0    0    2    0    0    0  538    0]
 [   1    0    0    0    0    0    0    2    0    0  577]]


In [ ]:
# save the model

torch.save(model.state_dict(), 'inceptionv3_fish_model.pth')